In [1]:
import sys
import torch
import torchvision.transforms as T
from IPython.display import Audio
import librosa
from src.Dataset import AudioMNIST
import custom_transforms as CT
from scipy import signal
from scipy.signal import ShortTimeFFT
from scipy.signal.windows import gaussian
from scipy.signal import stft, istft

In [2]:
sample_rate=16000
n_fft=512
root = '../Data'


In [3]:
from custom_transforms import TrimSilence, FixLength
from torch.utils.data import random_split,DataLoader

transforms = [TrimSilence(5), FixLength(16000)]
dataset = AudioMNIST(
    root,
    transform=T.Compose(transforms)
)
batch_size = 1
num_workers = 4
train_size = int(0.8 * len(dataset))
val_size = int(0.10 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(
    dataset,
    [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)
)

# Create dataloaders
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    persistent_workers=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    persistent_workers=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    persistent_workers=True
)


In [4]:
vqvae = torch.load('./saved_models/vqvae_monai.pth',weights_only=False).to(device='cuda:0')

2025-03-29 13:20:14.681057: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743268814.691711 1167352 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743268814.694967 1167352 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-29 13:20:14.707269: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/yagna/yagnaPythonEnv/lib/python3.12/site-packages/generative/networks/layers/vector_quantizer.py:86: FutureWarning: `to

In [5]:
next(iter(test_loader))[0].shape

torch.Size([1, 2, 128, 44])

In [6]:
x = vqvae.model.index_quantize(next(iter(test_loader))[0].to(device='cuda')).flatten(1)

In [7]:
x.shape

torch.Size([1, 352])

In [8]:
x = x.reshape(1,32,11)

In [9]:
x_hat = vqvae.model.decode_samples(x)

In [10]:
x_hat.shape

torch.Size([1, 2, 128, 44])

In [ ]:
import torch

nanogpt_cond = torch.load('saved_models/AudioMNISTDiffusion.pt',weights_only=False)
# Calculate the total number of parameters
total_params = sum(p.numel() for p in nanogpt_cond.parameters())

# Calculate the number of trainable parameters
trainable_params = sum(p.numel() for p in nanogpt_cond.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")

In [11]:
monai_cond = torch.load('saved_models/MONAI_Cond2_Transformer_epochs_50.pt',weights_only=False)

In [12]:
monai_cond

MonaiDecoderOnlyModel(
  (transformer): DecoderOnlyTransformer(
    (token_embeddings): Embedding(267, 256)
    (position_embeddings): AbsolutePositionalEmbedding(
      (embedding): Embedding(354, 256)
    )
    (embedding_dropout): Dropout(p=0.1, inplace=False)
    (blocks): ModuleList(
      (0-5): 6 x TransformerBlock(
        (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (attn): SABlock(
          (to_q): Linear(in_features=256, out_features=256, bias=False)
          (to_k): Linear(in_features=256, out_features=256, bias=False)
          (to_v): Linear(in_features=256, out_features=256, bias=False)
          (drop_weights): Dropout(p=0.0, inplace=False)
          (drop_output): Dropout(p=0.0, inplace=False)
          (out_proj): Linear(in_features=256, out_features=256, bias=True)
        )
        (norm3): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (mlp): MLPBlock(
          (linear1): Linear(in_features=256, out_features=1024, bias=Tr

In [ ]:
label = 5
context = torch.tensor([[256, 256 + 1 + label]], dtype=torch.long, device="cuda:1")
generated = monai_cond.generate(context, max_new_tokens=352)        
